In [2]:
import os
import pickle
import numpy as np
import pandas as pd

csv_pose_file = 'pose_urfd_3classes.csv'
save_path = 'urfd.pkl'

class_names = ['Not fall', 'Fall']
main_parts = ['LShoulder_x', 'LShoulder_y', 'RShoulder_x', 'RShoulder_y', 'LHip_x', 'LHip_y',
              'RHip_x', 'RHip_y']
main_idx_parts = [1, 2, 7, 8, -1]  # 1.5


# Params.
smooth_labels_step = 8
n_frames = 30
skip_frame = 1

annot = pd.read_csv(csv_pose_file)


In [3]:
num_rows = annot.shape[0]
print("Number of rows in annot:", num_rows)

Number of rows in annot: 2485


In [4]:
idx = annot.iloc[:, 2:-1][main_parts].isna().sum(1) > 0
idx = np.where(idx)[0]
print(idx)
annot = annot.drop(idx)
num_rows = annot.shape[0]
print("Number of rows in annot:", num_rows)


[1535 1673 2280]
Number of rows in annot: 2482


In [5]:
annot

,video,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,...,RKnee_x,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,label
0,fall-01,1,0.645445,0.259531,0.935683,0.691758,0.313563,0.886154,0.628078,0.321281,...,0.633867,0.560563,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,0
1,fall-01,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,...,0.631523,0.560563,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,0
2,fall-01,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,...,0.636250,0.560833,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853265,0
3,fall-01,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,...,0.635938,0.559688,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818946,0
4,fall-01,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,...,0.634500,0.565417,0.884659,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,fall-30,66,0.310078,0.875911,0.303253,0.277559,1.005990,0.089537,0.284063,0.971302,...,0.628770,0.858568,0.117682,0.485684,0.971302,0.156369,0.277559,0.988646,0.080249,2
2481,fall-30,67,0.319258,0.893490,0.232664,0.280586,0.996615,0.056626,0.280586,0.988021,...,0.622187,0.841927,0.136852,0.306367,0.902083,0.202636,0.280586,0.988021,0.133989,2
2482,fall-30,68,0.433301,0.910807,0.207741,0.447715,0.987682,0.064832,0.281953,0.968464,...,0.627891,0.833932,0.237444,0.303574,0.891589,0.377427,0.627891,0.853151,0.215639,2
2483,fall-30,69,0.423262,0.912214,0.122752,0.269453,1.018854,0.086390,0.287910,0.969635,...,0.620137,0.846589,0.140247,0.306367,0.895807,0.122043,0.620137,0.838385,0.100482,2


In [6]:
# One-Hot Labels.
label_onehot = pd.get_dummies(annot['label'])
annot = annot.drop('label', axis=1).join(label_onehot)
cols = label_onehot.columns.values

In [7]:
annot

,video,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,...,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1,2
0,fall-01,1,0.645445,0.259531,0.935683,0.691758,0.313563,0.886154,0.628078,0.321281,...,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,True,False,False
1,fall-01,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,...,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,True,False,False
2,fall-01,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,...,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853265,True,False,False
3,fall-01,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,...,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818946,True,False,False
4,fall-01,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,...,0.884659,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,fall-30,66,0.310078,0.875911,0.303253,0.277559,1.005990,0.089537,0.284063,0.971302,...,0.117682,0.485684,0.971302,0.156369,0.277559,0.988646,0.080249,False,False,True
2481,fall-30,67,0.319258,0.893490,0.232664,0.280586,0.996615,0.056626,0.280586,0.988021,...,0.136852,0.306367,0.902083,0.202636,0.280586,0.988021,0.133989,False,False,True
2482,fall-30,68,0.433301,0.910807,0.207741,0.447715,0.987682,0.064832,0.281953,0.968464,...,0.237444,0.303574,0.891589,0.377427,0.627891,0.853151,0.215639,False,False,True
2483,fall-30,69,0.423262,0.912214,0.122752,0.269453,1.018854,0.086390,0.287910,0.969635,...,0.140247,0.306367,0.895807,0.122043,0.620137,0.838385,0.100482,False,False,True


In [8]:
def scale_pose(xy):
    """
    Normalize pose points by scale with max/min value of each pose.
    xy : (frames, parts, xy) or (parts, xy)
    """
    if xy.ndim == 2:
        xy = np.expand_dims(xy, 0)
    xy_min = np.nanmin(xy, axis=1)
    xy_max = np.nanmax(xy, axis=1)
    for i in range(xy.shape[0]):
        xy[i] = ((xy[i] - xy_min[i]) / (xy_max[i] - xy_min[i])) * 2 - 1
    return xy.squeeze()
def seq_label_smoothing(labels, max_step=10):
    steps = 0
    remain_step = 0
    target_label = 0
    active_label = 0
    start_change = 0
    max_val = np.max(labels)
    min_val = np.min(labels)
    for i in range(labels.shape[0]):
        if remain_step > 0:
            if i >= start_change:
                labels[i][active_label] = max_val * remain_step / steps
                labels[i][target_label] = max_val * (steps - remain_step) / steps \
                    if max_val * (steps - remain_step) / steps else min_val
                remain_step -= 1
            continue

        diff_index = np.where(np.argmax(labels[i:i+max_step], axis=1) - np.argmax(labels[i]) != 0)[0]
        if len(diff_index) > 0:
            start_change = i + remain_step // 2
            steps = diff_index[0]
            remain_step = steps
            target_label = np.argmax(labels[i + remain_step])
            active_label = np.argmax(labels[i])
    return labels

In [9]:
feature_set = np.empty((0, n_frames, 14, 3))
labels_set = np.empty((0, len(cols)))
vid_list = annot['video'].unique()

In [10]:
feature_set

array([], shape=(0, 30, 14, 3), dtype=float64)

In [11]:
labels_set

array([], shape=(0, 3), dtype=float64)

In [12]:
annot

,video,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,...,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1,2
0,fall-01,1,0.645445,0.259531,0.935683,0.691758,0.313563,0.886154,0.628078,0.321281,...,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,True,False,False
1,fall-01,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,...,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,True,False,False
2,fall-01,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,...,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853265,True,False,False
3,fall-01,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,...,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818946,True,False,False
4,fall-01,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,...,0.884659,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,fall-30,66,0.310078,0.875911,0.303253,0.277559,1.005990,0.089537,0.284063,0.971302,...,0.117682,0.485684,0.971302,0.156369,0.277559,0.988646,0.080249,False,False,True
2481,fall-30,67,0.319258,0.893490,0.232664,0.280586,0.996615,0.056626,0.280586,0.988021,...,0.136852,0.306367,0.902083,0.202636,0.280586,0.988021,0.133989,False,False,True
2482,fall-30,68,0.433301,0.910807,0.207741,0.447715,0.987682,0.064832,0.281953,0.968464,...,0.237444,0.303574,0.891589,0.377427,0.627891,0.853151,0.215639,False,False,True
2483,fall-30,69,0.423262,0.912214,0.122752,0.269453,1.018854,0.086390,0.287910,0.969635,...,0.140247,0.306367,0.895807,0.122043,0.620137,0.838385,0.100482,False,False,True


In [13]:
vid = vid_list[0]

In [14]:
vid

'fall-01'

In [15]:
print(f'Process on: {vid}')
data = annot[annot['video'] == vid].reset_index(drop=True).drop(columns='video')


Process on: fall-01


In [16]:
data

,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,...,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1,2
0,1,0.645445,0.259531,0.935683,0.691758,0.313563,0.886154,0.628078,0.321281,0.942347,...,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,True,False,False
1,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,0.937608,...,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,True,False,False
2,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,0.943152,...,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853265,True,False,False
3,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,0.927087,...,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818946,True,False,False
4,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,0.960016,...,0.884659,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,156,0.395576,0.825326,0.270604,0.510010,0.816849,0.134676,0.427363,0.884661,0.366581,...,0.162887,0.408291,0.842279,0.244469,0.408291,0.842279,0.271072,False,False,True
151,157,0.393281,0.815026,0.313128,0.437373,0.873815,0.233434,0.424775,0.882214,0.292854,...,0.159534,0.412178,0.840221,0.337166,0.412178,0.840221,0.313878,False,False,True
152,158,0.393594,0.832812,0.349999,0.437891,0.883438,0.178317,0.431563,0.891875,0.201614,...,0.175705,0.412578,0.841250,0.332188,0.412578,0.841250,0.278103,False,False,True
153,159,0.397480,0.825130,0.346877,0.435801,0.876224,0.177355,0.429414,0.884740,0.358989,...,0.182200,0.410254,0.842161,0.275296,0.410254,0.842161,0.307140,False,False,True


In [17]:
cols

array([0, 1, 2], dtype=int64)

In [18]:

# Label Smoothing.
esp = 0.1
data[cols] = data[cols] * (1 - esp) + (1 - data[cols]) * esp / (len(cols) - 1)
data[cols] = seq_label_smoothing(data[cols].values, smooth_labels_step)



In [19]:
data

,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,...,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,0,1,2
0,1,0.645445,0.259531,0.935683,0.691758,0.313563,0.886154,0.628078,0.321281,0.942347,...,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590,0.90,0.05,0.05
1,2,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,0.937608,...,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271,0.90,0.05,0.05
2,3,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,0.943152,...,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853265,0.90,0.05,0.05
3,4,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,0.927087,...,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818946,0.90,0.05,0.05
4,5,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,0.960016,...,0.884659,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341,0.90,0.05,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,156,0.395576,0.825326,0.270604,0.510010,0.816849,0.134676,0.427363,0.884661,0.366581,...,0.162887,0.408291,0.842279,0.244469,0.408291,0.842279,0.271072,0.05,0.05,0.90
151,157,0.393281,0.815026,0.313128,0.437373,0.873815,0.233434,0.424775,0.882214,0.292854,...,0.159534,0.412178,0.840221,0.337166,0.412178,0.840221,0.313878,0.05,0.05,0.90
152,158,0.393594,0.832812,0.349999,0.437891,0.883438,0.178317,0.431563,0.891875,0.201614,...,0.175705,0.412578,0.841250,0.332188,0.412578,0.841250,0.278103,0.05,0.05,0.90
153,159,0.397480,0.825130,0.346877,0.435801,0.876224,0.177355,0.429414,0.884740,0.358989,...,0.182200,0.410254,0.842161,0.275296,0.410254,0.842161,0.307140,0.05,0.05,0.90


In [20]:
frames = data['frame'].values
frames_set = []
fs = [0]

In [21]:
frames

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 115, 117, 118, 119, 120, 121,
       122, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160],
      dtype=int64)

In [22]:
for i in range(1, len(frames)):
    # print(i, frames[i] < frames[i-1] + 10)
    if frames[i] < frames[i-1] + 10:
        fs.append(i)
    else:
        print(i, "********")
        frames_set.append(fs)
        fs = [i]
frames_set.append(fs)

In [23]:
fs = frames_set[0]

In [24]:
fs

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154]

In [27]:
data.shape

(155, 43)

In [28]:
data.iloc[fs, 1:-len(cols)]

,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,LElbow_x,...,LKnee_s,RKnee_x,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s
0,0.645445,0.259531,0.935683,0.691758,0.313563,0.886154,0.628078,0.321281,0.942347,0.714914,...,0.843394,0.633867,0.560563,0.875167,0.697547,0.645469,0.655076,0.645445,0.645469,0.869590
1,0.643102,0.259531,0.925543,0.689414,0.313563,0.885889,0.625734,0.321281,0.937608,0.712570,...,0.861147,0.631523,0.560563,0.849684,0.695203,0.645469,0.673745,0.643102,0.645469,0.861271
2,0.647969,0.256146,0.944614,0.688984,0.310833,0.891619,0.624531,0.318646,0.943152,0.712422,...,0.832305,0.636250,0.560833,0.863793,0.694844,0.646771,0.793520,0.642109,0.646771,0.853265
3,0.647422,0.261094,0.952466,0.687617,0.314688,0.863852,0.624453,0.322344,0.927087,0.710586,...,0.843946,0.635938,0.559688,0.876088,0.699102,0.643906,0.647590,0.647422,0.643906,0.818946
4,0.646313,0.258292,0.947201,0.687656,0.313417,0.897403,0.622688,0.321292,0.960016,0.711281,...,0.843397,0.634500,0.565417,0.884659,0.699469,0.652042,0.716619,0.646313,0.644167,0.852341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,0.395576,0.825326,0.270604,0.510010,0.816849,0.134676,0.427363,0.884661,0.366581,0.421006,...,0.323798,0.433721,0.859232,0.162887,0.408291,0.842279,0.244469,0.408291,0.842279,0.271072
151,0.393281,0.815026,0.313128,0.437373,0.873815,0.233434,0.424775,0.882214,0.292854,0.380684,...,0.209731,0.437373,0.873815,0.159534,0.412178,0.840221,0.337166,0.412178,0.840221,0.313878
152,0.393594,0.832812,0.349999,0.437891,0.883438,0.178317,0.431563,0.891875,0.201614,0.418906,...,0.355079,0.437891,0.875000,0.175705,0.412578,0.841250,0.332188,0.412578,0.841250,0.278103
153,0.397480,0.825130,0.346877,0.435801,0.876224,0.177355,0.429414,0.884740,0.358989,0.423027,...,0.179731,0.435801,0.876224,0.182200,0.410254,0.842161,0.275296,0.410254,0.842161,0.307140


In [29]:
data.iloc[fs, 1:-len(cols)].values.reshape(-1, 13, 3)[0]

array([[0.64544535, 0.25953126, 0.93568349],
       [0.6917578 , 0.31356251, 0.88615358],
       [0.6280781 , 0.32128128, 0.94234747],
       [0.71491408, 0.38303125, 0.89677685],
       [0.61650002, 0.39075002, 0.91544002],
       [0.72070312, 0.45250002, 0.84727484],
       [0.60492194, 0.45250002, 0.90431345],
       [0.68017972, 0.46793753, 0.8063634 ],
       [0.6338672 , 0.46793753, 0.80632496],
       [0.6917578 , 0.56056255, 0.84339386],
       [0.6338672 , 0.56056255, 0.87516731],
       [0.69754684, 0.64546883, 0.65507621],
       [0.64544535, 0.64546883, 0.86958963]])

In [30]:
xys = data.iloc[fs, 1:-len(cols)].values.reshape(-1, 13, 3)

In [31]:
xys.shape

(155, 13, 3)

In [32]:
xys[0]

array([[0.64544535, 0.25953126, 0.93568349],
       [0.6917578 , 0.31356251, 0.88615358],
       [0.6280781 , 0.32128128, 0.94234747],
       [0.71491408, 0.38303125, 0.89677685],
       [0.61650002, 0.39075002, 0.91544002],
       [0.72070312, 0.45250002, 0.84727484],
       [0.60492194, 0.45250002, 0.90431345],
       [0.68017972, 0.46793753, 0.8063634 ],
       [0.6338672 , 0.46793753, 0.80632496],
       [0.6917578 , 0.56056255, 0.84339386],
       [0.6338672 , 0.56056255, 0.87516731],
       [0.69754684, 0.64546883, 0.65507621],
       [0.64544535, 0.64546883, 0.86958963]])

In [33]:
xys[:, :, :2] = scale_pose(xys[:, :, :2])

In [34]:
xys[0]

array([[-3.00000103e-01, -1.00000000e+00,  9.35683489e-01],
       [ 4.99999485e-01, -7.20000037e-01,  8.86153579e-01],
       [-6.00001236e-01, -6.79999932e-01,  9.42347467e-01],
       [ 9.00000309e-01, -3.60000173e-01,  8.96776855e-01],
       [-8.00000618e-01, -3.20000068e-01,  9.15440023e-01],
       [ 1.00000000e+00, -1.54441156e-07,  8.47274840e-01],
       [-1.00000000e+00, -1.54441156e-07,  9.04313445e-01],
       [ 3.00000103e-01,  7.99999012e-02,  8.06363404e-01],
       [-5.00000515e-01,  7.99999012e-02,  8.06324959e-01],
       [ 4.99999485e-01,  5.59999926e-01,  8.43393862e-01],
       [-5.00000515e-01,  5.59999926e-01,  8.75167310e-01],
       [ 5.99999176e-01,  1.00000000e+00,  6.55076206e-01],
       [-3.00000103e-01,  1.00000000e+00,  8.69589627e-01]])

In [35]:
xys = np.concatenate((xys, np.expand_dims((xys[:, 1, :] + xys[:, 2, :]) / 2, 1)), axis=1)


In [36]:
xys[0]

array([[-3.00000103e-01, -1.00000000e+00,  9.35683489e-01],
       [ 4.99999485e-01, -7.20000037e-01,  8.86153579e-01],
       [-6.00001236e-01, -6.79999932e-01,  9.42347467e-01],
       [ 9.00000309e-01, -3.60000173e-01,  8.96776855e-01],
       [-8.00000618e-01, -3.20000068e-01,  9.15440023e-01],
       [ 1.00000000e+00, -1.54441156e-07,  8.47274840e-01],
       [-1.00000000e+00, -1.54441156e-07,  9.04313445e-01],
       [ 3.00000103e-01,  7.99999012e-02,  8.06363404e-01],
       [-5.00000515e-01,  7.99999012e-02,  8.06324959e-01],
       [ 4.99999485e-01,  5.59999926e-01,  8.43393862e-01],
       [-5.00000515e-01,  5.59999926e-01,  8.75167310e-01],
       [ 5.99999176e-01,  1.00000000e+00,  6.55076206e-01],
       [-3.00000103e-01,  1.00000000e+00,  8.69589627e-01],
       [-5.00008752e-02, -6.99999985e-01,  9.14250523e-01]])

In [37]:
xys.shape

(155, 14, 3)

In [42]:
xys[:, :, -1]

array([[0.93568349, 0.88615358, 0.94234747, ..., 0.65507621, 0.86958963,
        0.91425052],
       [0.92554295, 0.88588858, 0.937608  , ..., 0.67374468, 0.86127126,
        0.91174829],
       [0.94461358, 0.89161915, 0.94315189, ..., 0.79351979, 0.85326546,
        0.91738552],
       ...,
       [0.34999943, 0.17831683, 0.20161387, ..., 0.3321884 , 0.27810311,
        0.18996535],
       [0.34687677, 0.17735535, 0.35898915, ..., 0.27529642, 0.30713952,
        0.26817225],
       [0.41118038, 0.16861534, 0.15229698, ..., 0.31029561, 0.41195568,
        0.16045616]])

In [38]:
scr = xys[:, :, -1].copy()

In [39]:
scr 

array([[0.93568349, 0.88615358, 0.94234747, ..., 0.65507621, 0.86958963,
        0.91425052],
       [0.92554295, 0.88588858, 0.937608  , ..., 0.67374468, 0.86127126,
        0.91174829],
       [0.94461358, 0.89161915, 0.94315189, ..., 0.79351979, 0.85326546,
        0.91738552],
       ...,
       [0.34999943, 0.17831683, 0.20161387, ..., 0.3321884 , 0.27810311,
        0.18996535],
       [0.34687677, 0.17735535, 0.35898915, ..., 0.27529642, 0.30713952,
        0.26817225],
       [0.41118038, 0.16861534, 0.15229698, ..., 0.31029561, 0.41195568,
        0.16045616]])

In [43]:
scr[:, main_idx_parts] = np.minimum(scr[:, main_idx_parts] * 1.5, 1.0)

In [45]:
scr

array([[0.93568349, 1.        , 1.        , ..., 0.65507621, 0.86958963,
        1.        ],
       [0.92554295, 1.        , 1.        , ..., 0.67374468, 0.86127126,
        1.        ],
       [0.94461358, 1.        , 1.        , ..., 0.79351979, 0.85326546,
        1.        ],
       ...,
       [0.34999943, 0.26747525, 0.30242081, ..., 0.3321884 , 0.27810311,
        0.28494803],
       [0.34687677, 0.26603302, 0.53848372, ..., 0.27529642, 0.30713952,
        0.40225837],
       [0.41118038, 0.25292301, 0.22844546, ..., 0.31029561, 0.41195568,
        0.24068424]])

In [54]:
scr = scr.mean(1)


In [56]:
src

array([0.91019398, 0.91241001, 0.91724952, 0.90657711, 0.91334903,
       0.91724865, 0.91219006, 0.9124426 , 0.90535212, 0.91298749,
       0.91710692, 0.91181025, 0.91621791, 0.92330284, 0.9181954 ,
       0.91469467, 0.90656852, 0.9148121 , 0.91995053, 0.9229098 ,
       0.91275406, 0.92050546, 0.91637344, 0.91951028, 0.92638569,
       0.92024072, 0.91267933, 0.92293876, 0.92416634, 0.92545996,
       0.92974682, 0.93456282, 0.92864957, 0.93473545, 0.93365721,
       0.92915732, 0.93444122, 0.93125529, 0.9283269 , 0.93735515,
       0.93886287, 0.93617313, 0.94181029, 0.94378556, 0.9438099 ,
       0.94041353, 0.9449416 , 0.94549755, 0.93376287, 0.92025221,
       0.92620009, 0.92922748, 0.9350987 , 0.92981297, 0.93109461,
       0.93222306, 0.91924704, 0.91449426, 0.91750773, 0.90559742,
       0.91426388, 0.9164909 , 0.92094613, 0.90572288, 0.90702177,
       0.91505951, 0.92131355, 0.91542559, 0.92281127, 0.91665673,
       0.92459317, 0.93195374, 0.93393966, 0.94669562, 0.94490

In [57]:
# Targets.
lb = data.iloc[fs, -len(cols):].values



In [58]:
lb

array([[0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.9       , 0.05      , 0.05      ],
       [0.

In [59]:
# Apply points score mean to all labels.
lb = lb * scr[:, None]

In [60]:
lb

array([[0.81917458, 0.0455097 , 0.0455097 ],
       [0.82116901, 0.0456205 , 0.0456205 ],
       [0.82552457, 0.04586248, 0.04586248],
       [0.8159194 , 0.04532886, 0.04532886],
       [0.82201413, 0.04566745, 0.04566745],
       [0.82552379, 0.04586243, 0.04586243],
       [0.82097106, 0.0456095 , 0.0456095 ],
       [0.82119834, 0.04562213, 0.04562213],
       [0.8148169 , 0.04526761, 0.04526761],
       [0.82168874, 0.04564937, 0.04564937],
       [0.82539623, 0.04585535, 0.04585535],
       [0.82062922, 0.04559051, 0.04559051],
       [0.82459612, 0.0458109 , 0.0458109 ],
       [0.83097256, 0.04616514, 0.04616514],
       [0.82637586, 0.04590977, 0.04590977],
       [0.8232252 , 0.04573473, 0.04573473],
       [0.81591167, 0.04532843, 0.04532843],
       [0.82333089, 0.0457406 , 0.0457406 ],
       [0.82795548, 0.04599753, 0.04599753],
       [0.83061882, 0.04614549, 0.04614549],
       [0.82147865, 0.0456377 , 0.0456377 ],
       [0.82845491, 0.04602527, 0.04602527],
       [0.

In [65]:
xys[1:1+30].shape

(30, 14, 3)

In [66]:
xys[1:1+30][None, ...].shape

(1, 30, 14, 3)